# mHSP2: Example Modifications for Standalone to Coupled

This notebook provides an example of the modifications required to transition an **mHSP2** standalone model to a coupled model.

Minimal changes are required for an **mHSP2** model. The primary changes are to remove the external inflows. Normally, external inflows represent spring discharge or baseflow in an **mHSP2**model. In coupled mode, these external inflows come from the **pyMF6** model. Consequently, the external inflows are no longer specified as part of **mHSP2** model inputs.

For this example, model there is one external inflow time series, *TS136*. It is removed from the model inputs in this notebook. After this change, the **mHSP2** model is ready for coupled mode simulation.

## Parameters and Imports

In [1]:
import os
import h5py
import numpy as np
import pandas as pd
from IPython.display import display, HTML
import matplotlib as mpl
import matplotlib.pyplot as plt
import datetime as dt

In [2]:
H_PATH = r'C:\Working\Test_Models\modCP_mHSP2\DC_CalibmHSP2.h5'

## Load the HDF5 File

In [3]:
store = pd.HDFStore( H_PATH )

In [4]:
exSources = store.get( key= r'/CONTROL/EXT_SOURCES/' )

In [5]:
exSources

,SVOL,SVOLNO,SMEMSB,SSYST,MFACTOR,TRAN,TVOL,TMEMN,TMEMSB,TVOLNO,AUX_TS,COMMENTS
0,*,TS102,31,ENGL,1.0,SAME,RCHRES,PREC,,R001,,
1,*,TS103,31,ENGL,1.0,SAME,RCHRES,POTEV,,R001,,
2,*,TS104,31,ENGL,1.0,SAME,RCHRES,PREC,,R002,,
3,*,TS105,31,ENGL,1.0,SAME,RCHRES,POTEV,,R002,,
4,*,TS106,31,ENGL,1.0,SAME,RCHRES,PREC,,R003,,
5,*,TS107,31,ENGL,1.0,SAME,RCHRES,POTEV,,R003,,
6,*,TS108,31,ENGL,1.0,SAME,RCHRES,PREC,,R004,,
7,*,TS109,31,ENGL,1.0,SAME,RCHRES,POTEV,,R004,,
8,*,TS111,31,ENGL,1.0,SAME,RCHRES,POTEV,,R005,,
9,*,TS110,31,ENGL,1.0,SAME,RCHRES,PREC,,R005,,


## Remove TS136

In [6]:
exSources.drop(index=58, inplace=True)

In [7]:
exSources

,SVOL,SVOLNO,SMEMSB,SSYST,MFACTOR,TRAN,TVOL,TMEMN,TMEMSB,TVOLNO,AUX_TS,COMMENTS
0,*,TS102,31,ENGL,1.0,SAME,RCHRES,PREC,,R001,,
1,*,TS103,31,ENGL,1.0,SAME,RCHRES,POTEV,,R001,,
2,*,TS104,31,ENGL,1.0,SAME,RCHRES,PREC,,R002,,
3,*,TS105,31,ENGL,1.0,SAME,RCHRES,POTEV,,R002,,
4,*,TS106,31,ENGL,1.0,SAME,RCHRES,PREC,,R003,,
5,*,TS107,31,ENGL,1.0,SAME,RCHRES,POTEV,,R003,,
6,*,TS108,31,ENGL,1.0,SAME,RCHRES,PREC,,R004,,
7,*,TS109,31,ENGL,1.0,SAME,RCHRES,POTEV,,R004,,
8,*,TS111,31,ENGL,1.0,SAME,RCHRES,POTEV,,R005,,
9,*,TS110,31,ENGL,1.0,SAME,RCHRES,PREC,,R005,,


In [8]:
store.put( key=r'/CONTROL/EXT_SOURCES/', value=exSources, format='f', data_columns=True )

In [9]:
store.close()